In [72]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpAffineExpression
import numpy as np
import pulp

In [29]:
import pulp as pl

In [32]:
solver_list = pl.listSolvers(onlyAvailable=True)
solver_list

['PULP_CBC_CMD']

In [15]:
foods = pd.read_csv('./Data_formatted/nv_df.csv', index_col='Unnamed: 0')
constraints = pd.read_csv('./Data_formatted/constraints.csv', index_col = 'Unnamed: 0')

In [41]:
foods.head()
constraints.head()
constraints.loc['calories',:] = [1500,2500]

In [51]:
protein_calories = foods['protein']/foods['calories'].sort_values(ascending=False).copy()

In [62]:

# selecting high protein to calorie ratio foods
protein_foods = foods.loc[protein_calories.sort_values(ascending=False)[0:40].index,:]\
.query("calories > 50")

In [65]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in protein_foods.index],
}

data_foods.update({col: protein_foods[col] for col in protein_foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Integer')

# Define the objective function to minimize the total number of servings
problem += lpSum(food_vars[food] for food in df_foods['Food'])

# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ea5f81a947d140a2b1321171f1d78d40-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ea5f81a947d140a2b1321171f1d78d40-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 74 COLUMNS
At line 1575 RHS
At line 1645 BOUNDS
At line 1670 ENDATA
Problem MODEL has 69 rows, 24 columns and 1428 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: -1
No solution found.


Nex I will solve to minimize calories since minimizing for serving size has no solution.

In [66]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
problem += lpSum(food_vars[food]s for food in df_foods['Food'])

# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ff0f0c9c235c463c8ca5cf394864d59f-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ff0f0c9c235c463c8ca5cf394864d59f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 74 COLUMNS
At line 6955 RHS
At line 7025 BOUNDS
At line 7026 ENDATA
Problem MODEL has 69 rows, 117 columns and 6763 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
Perturbing problem by 0.001% of 43.417997 - largest nonzero change 8.4803796e-05 ( 0.015683019%) - largest zero change 0
0  Obj 0 Primal inf 14279.833 (43)
11  Obj 75032.353 Pri

Looking to minimize calories:

In [73]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
ae_calories = LpAffineExpression([(food_vars[food],df_foods.loc[food,'calories']) for food in df_foods['Food']])
problem += ae_calories
# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/f78241024e06443a8434c0a69bfdbfb1-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/f78241024e06443a8434c0a69bfdbfb1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 74 COLUMNS
At line 6955 RHS
At line 7025 BOUNDS
At line 7026 ENDATA
Problem MODEL has 69 rows, 117 columns and 6763 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 14279.833 (43)
10  Obj 5639150.6 Primal inf 7633460.6 (24)
Primal infeasible - objective value 5639150.6
PrimalInfeasible objective 5639150.603 - 10 iteratio

In [ ]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
ae_calories = LpAffineExpression([(food_vars[food],df_foods.loc[food,'calories']) for food in df_foods['Food']])
problem += ae_calories
# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")